In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-26 03:02:17--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.91MB/s    in 0.2s    

2022-09-26 03:02:17 (4.91 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# Start SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://misse-bucket.s3.us-west-2.amazonaws.com/amazon_reviews_multilingual_US_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_multilingual_US_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   53096384| R63J84G1LOX6R|1563890119|     763187671|The Sandman Vol. ...|           Books|          4|            0|          1|   N|                N|ignore the review...|this is the first...| 1995-08-13|
|         US|   53096399|R1BALOA11Z06MT|1559947608|     381720534|The 22 Immutable ...|           Books|          4|    

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df = spark.read.csv(SparkFiles.get("amazon_reviews_multilingual_US_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   53096384| R63J84G1LOX6R|1563890119|     763187671|The Sandman Vol. ...|           Books|          4|            0|          1|   N|                N|ignore the review...|this is the first...| 1995-08-13|
|         US|   53096399|R1BALOA11Z06MT|1559947608|     381720534|The 22 Immutable ...|           Books|          4|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"]).drop_duplicates()
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100AXNMFVACPY|          2|            4|          7|   N|                N|
|R100DRQ6Z1C6D6|          5|            0|          0|   N|                Y|
|R100LZZVTVLB08|          1|            7|         16|   N|                N|
|R100UUK035JD4E|          5|            0|          0|   N|                Y|
|R100W7NQZM08J5|          4|            1|          1|   N|                Y|
|R100ZB7HIXV0OZ|          4|            0|          0|   N|                N|
|R1012NLMFN5M2V|          1|            0|          1|   N|                Y|
|R101AGLAQ7TCAF|          5|            0|          0|   N|                Y|
|R101DTK4EQVPA7|          5|            0|          1|   N|                Y|
|R101KI89GF1VUT|          5|            0|          0|   N|     

In [7]:
# Create the total_votes DataFrame
total_votes_df = vine_df.filter("total_votes >=20")
total_votes_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2GEGEK0MS1XYM|          4|           33|         39|   N|                Y|
|R2Q75G45MBFFJJ|          1|            8|         31|   N|                N|
|R2SX2FJQJNA6EV|          5|           34|         35|   N|                N|
|R1Z17CVFBXSY5R|          1|           16|         36|   N|                N|
|R37D6LJ0IEDGGW|          5|          117|        122|   N|                N|
|R22UI1T5VE6LPR|          5|           82|         88|   N|                N|
| RY2CNB5SBU6UH|          5|           21|         24|   N|                N|
|R3NWAZ5RPHWCH3|          1|           28|         30|   N|                N|
|R2LB86QZF54NQP|          1|            3|         27|   N|                N|
|R2OJ5QCBMH58YS|          2|            2|         21|   N|     

In [40]:
# Filter dataset and create total_votes DataFrame where total_votes greater than or equal to 20
total_votes_df = vine_df.filter(vine_df["total_votes"] >=20)
total_votes_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2GEGEK0MS1XYM|          4|           33|         39|   N|                Y|
|R2Q75G45MBFFJJ|          1|            8|         31|   N|                N|
|R2SX2FJQJNA6EV|          5|           34|         35|   N|                N|
|R1Z17CVFBXSY5R|          1|           16|         36|   N|                N|
|R37D6LJ0IEDGGW|          5|          117|        122|   N|                N|
|R22UI1T5VE6LPR|          5|           82|         88|   N|                N|
| RY2CNB5SBU6UH|          5|           21|         24|   N|                N|
|R3NWAZ5RPHWCH3|          1|           28|         30|   N|                N|
|R2LB86QZF54NQP|          1|            3|         27|   N|                N|
|R2OJ5QCBMH58YS|          2|            2|         21|   N|     

In [54]:
# Filter total_votes DataFrame and divide helpful_votes by total_votes which is equal to or greater than 50%
helpful_votes_df = total_votes_df.filter(total_votes_df['helpful_votes']/total_votes_df['total_votes'] >= 0.5)
helpful_votes_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2GEGEK0MS1XYM|          4|           33|         39|   N|                Y|
|R2SX2FJQJNA6EV|          5|           34|         35|   N|                N|
|R37D6LJ0IEDGGW|          5|          117|        122|   N|                N|
|R22UI1T5VE6LPR|          5|           82|         88|   N|                N|
| RY2CNB5SBU6UH|          5|           21|         24|   N|                N|
|R3NWAZ5RPHWCH3|          1|           28|         30|   N|                N|
| RDXNOF27OF9DE|          5|           43|         47|   N|                N|
|R2X36RX301LIAH|          5|           26|         29|   N|                N|
|R23M1NGU7MLAJ7|          4|           30|         32|   N|                N|
|R35M6B2YNBSYTI|          3|           19|         35|   N|     

In [55]:
# Filter helpful_votes DataFrame and retrieve reviews that were paid
paid_df = helpful_votes_df.filter(helpful_votes_df["vine"] == 'Y')
paid_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2OUE8JITCXDZW|          4|           43|         46|   Y|                N|
|R1O29Y6ITFVYZC|          4|           38|         41|   Y|                N|
|R3CHEUY88EA4QH|          4|           54|         78|   Y|                N|
| R8N3SFC5CGAR4|          5|           17|         21|   Y|                N|
|  RV8FVXVWTEAX|          4|           97|        107|   Y|                N|
|R34IMC0WG84MMB|          5|           34|         39|   Y|                N|
|R21P35ZHD7MCY4|          1|           14|         25|   Y|                N|
|R3OKAQQU6FGJQU|          5|          318|        344|   Y|                N|
|R2N88D7RW8J5V9|          2|           28|         47|   Y|                N|
|R2UOQQOSMCCJ1U|          5|           20|         26|   Y|     

In [46]:
# Filter helpful_votes DataFrame and retrieve reviews that were unpaid
unpaid_df = helpful_votes_df.filter(helpful_votes_df["vine"] == 'N')
unpaid_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2GEGEK0MS1XYM|          4|           33|         39|   N|                Y|
|R2SX2FJQJNA6EV|          5|           34|         35|   N|                N|
|R37D6LJ0IEDGGW|          5|          117|        122|   N|                N|
|R22UI1T5VE6LPR|          5|           82|         88|   N|                N|
| RY2CNB5SBU6UH|          5|           21|         24|   N|                N|
|R3NWAZ5RPHWCH3|          1|           28|         30|   N|                N|
| RDXNOF27OF9DE|          5|           43|         47|   N|                N|
|R2X36RX301LIAH|          5|           26|         29|   N|                N|
|R23M1NGU7MLAJ7|          4|           30|         32|   N|                N|
|R35M6B2YNBSYTI|          3|           19|         35|   N|     

In [49]:
# Determine the total numnber of reviews and paid 5 stars
paid_start_5 = paid_df[paid_df["star_rating"]==5].count()
all_count_df_paid = paid_df.count()

In [50]:
# Determine the total numnber of reviews and unpaid 5 stars
unpaid_start_5 = unpaid_df[unpaid_df["star_rating"]==5].count()
all_count_df_unpaid = unpaid_df.count()

In [51]:
# print percentage of paid five stars by total number of reviews of paid
print(float(paid_start_5)/float(all_count_df_paid))

0.4438305709023941


In [52]:
# print percentage of unpaid five stars by total number of reviews of unpaid
print(float(unpaid_start_5)/float(all_count_df_unpaid))

0.4627093251066018
